In [130]:
import pandas as pd
import numpy as np
import json
import os
import glob
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import math

In [131]:
DATA = '2024-11-27'

In [132]:
DIRETORIO_DADOS = "../data/baterias/"

In [133]:
N_MIN = 8
P_BASE = 400
T_CRES = 8
P_MIN = 10
FRAC_TT = 0.07
P_MIN_TT = 0
FRAC_MVC = 0.12
P_VC = 5

In [134]:
def caulcarPCC(pos, numero_participantes):
  if pos > numero_participantes: 
      return 0
  p1_cc = P_BASE + T_CRES * (numero_participantes - N_MIN)
  return round((-1 * math.log(pos) * ((p1_cc - P_MIN) / math.log(numero_participantes)) + p1_cc), 2)

def caulcarPMV(numero_participantes):
  p1_cc = P_BASE + T_CRES * (numero_participantes - N_MIN)
  return round(100 * FRAC_MVC * p1_cc) / 100


def caulcarPCTT(pos, numero_participantes):
  if pos > numero_participantes: 
    return 0
  p1_cc = P_BASE + T_CRES * (numero_participantes - N_MIN)
  p1_ctt = p1_cc * FRAC_TT
  return round(max((-1 * math.log(pos) * p1_ctt / math.log((numero_participantes + 1) / 2)) + p1_ctt, 0), 2)

def caulcarPVC(voltas):
  return P_VC * voltas

In [135]:
def buscar_corridas(data_str=None):
    
    if data_str is None:
        data_entrada = datetime.now()
    else:
        data_entrada = datetime.strptime(data_str, "%Y-%m-%d")

    data_inicio = data_entrada - relativedelta(years=1)
    
    corridas = []
    
    padrao_arquivos = os.path.join(DIRETORIO_DADOS, "*.json")
    arquivos = glob.glob(padrao_arquivos)
    
    for arquivo in arquivos:
        nome_arquivo = os.path.basename(arquivo)
        
        try:
            data_arquivo_str = nome_arquivo[:8]
            data_arquivo = datetime.strptime(data_arquivo_str, "%Y%m%d")
            
            if data_inicio <= data_arquivo <= data_entrada:
                with open(arquivo, "r") as f:
                    corrida = json.load(f)
                    corridas.append(corrida)
        except ValueError:
            continue
    
    return corridas

def listar_todos_pilotos(corridas):
    participantes_corridas = []
    for corrida in corridas:
        participantes = set()
        if corrida['corrida'] is not None:
            [participantes.add(piloto['NOME']) for piloto in corrida['corrida']]
        if corrida['tomada_de_tempo'] is not None:
            [participantes.add(piloto['NOME']) for piloto in corrida['tomada_de_tempo']]
        participantes_corridas.append(participantes)
    return participantes_corridas

def selecionar_pilotos_qualificados(lista_de_sets, proporcao=0):
    contador = {}

    with open('../data/ignorar.json', 'r') as file:
        ignorar = json.load(file)

    for conjunto in lista_de_sets:
        for nome in conjunto:
            if nome in contador:
                contador[nome] += 1
            else:
                contador[nome] = 1

    numero_sets = len(lista_de_sets)
    minimo_aparicoes = numero_sets * proporcao

    pilotos_qualificados = [nome for nome, contagem in contador.items() if contagem >= minimo_aparicoes and nome not in ignorar]

    return pilotos_qualificados

def calcular_POS(list):
    list.sort(key=lambda piloto: int(piloto['POS']))
    for i, piloto in enumerate(list):
        piloto["POS"] = i + 1

def calcular_pontos_por_corrida(corridas, pilotos_qualificados):
    pontuacoes_corridas = {
        'total': [],
        'baterias': []
    }
    for corrida in corridas:

        print(corrida)

        participantes = set()
        if corrida['corrida'] is not None:
            corrida_filtrado = [piloto for piloto in corrida['corrida'] if piloto['NOME'] in pilotos_qualificados]
        if corrida['tomada_de_tempo'] is not None:
            tomada_de_tempo_filtrado = [piloto for piloto in corrida['tomada_de_tempo'] if piloto['NOME'] in pilotos_qualificados]

        print(" --- ")
        print(corrida_filtrado)
        print(tomada_de_tempo_filtrado)
        print(" --- ")

        pontuacao_corrida = {
            'bateria': corrida['bateria'],
            'pilotos': [],
            'numero_participantes_corrida': len(corrida_filtrado),
            'numero_participantes_tomada_de_tempo': len(tomada_de_tempo_filtrado)
        }

        [participantes.add(piloto['NOME']) for piloto in corrida_filtrado]
        [participantes.add(piloto['NOME']) for piloto in tomada_de_tempo_filtrado]

        calcular_POS(corrida_filtrado)
        calcular_POS(tomada_de_tempo_filtrado)

        MELHOR_VOLTA = min(corrida_filtrado, key=lambda piloto: piloto["TMV"])["TMV"]


        print(participantes)

        for p in participantes:
            p_corrida = next((piloto for piloto in corrida_filtrado if piloto["NOME"] == p), None)
            p_tomada_de_tempo = next((piloto for piloto in tomada_de_tempo_filtrado if piloto["NOME"] == p), None)



            piloto = {'NOME': p,
                      'POS_CORRIDA': p_corrida['POS'] if p_corrida else None,
                      'POS_TOMADA_DE_TEMPO': p_tomada_de_tempo['POS'] if p_tomada_de_tempo else None,
                      'MVC': p_corrida['TMV'] == MELHOR_VOLTA,
                      'VC': (int(p_corrida['VLTS']) if p_corrida else 0) + (int(p_tomada_de_tempo['VLTS']) if p_tomada_de_tempo else 0)
                      }
        
            piloto['PCC'] = caulcarPCC(piloto['POS_CORRIDA'], pontuacao_corrida['numero_participantes_corrida']) if piloto['POS_CORRIDA'] else 0
            piloto['PMV'] = caulcarPMV(pontuacao_corrida['numero_participantes_corrida']) if piloto['MVC'] == True else 0
            piloto['PCTT'] = caulcarPCTT(piloto['POS_TOMADA_DE_TEMPO'], pontuacao_corrida['numero_participantes_tomada_de_tempo']) if piloto['POS_TOMADA_DE_TEMPO'] else 0
            piloto['PVC'] = caulcarPVC(piloto['VC'])
            piloto['PTOTAL'] = round(piloto['PCC'] + piloto['PMV'] + piloto['PCTT'] + piloto['PVC'], 2)

            pontuacao_corrida['pilotos'].append(piloto)

        pontuacoes_corridas['baterias'].append(pontuacao_corrida)

    for piloto in pilotos_qualificados:
        PTOTAL = 0
        for bateria in pontuacoes_corridas['baterias']:
            p =  next((p for p in bateria['pilotos'] if p["NOME"] == piloto), None)
            PTOTAL += p['PTOTAL'] if p else 0
    
        pontuacoes_corridas['total'].append({'NOME': piloto, 'PTOTAL': round(PTOTAL, 2)})
        
    pontuacoes_corridas['total'].sort(key=lambda piloto: int(piloto['PTOTAL']), reverse = True)
    for i, piloto in enumerate(pontuacoes_corridas['total']):
        piloto["POS"] = i + 1

    return pontuacoes_corridas

In [136]:
corridas = buscar_corridas(DATA)
participantes_corridas = listar_todos_pilotos(corridas)
pilotos_qualificados = selecionar_pilotos_qualificados(participantes_corridas)
ranking = calcular_pontos_por_corrida(corridas, pilotos_qualificados)

{'url': 'https://www.mylaptime.com/laptime/clientes/9T00V814PC29X0160812G781W/results/r3.html?evt=11611&epg=8993', 'bateria': '202403201815', 'corrida': [{'POS': '1', '#': '53', 'CAT': 'INDOOR', 'NOME': 'BRENER SILVA', 'MV': '9', 'TMV': '01:49.029', 'VLTS': '12', 'TT': '00:22:39.959', 'DL': '---', 'DA': '---', 'VM': 36.56, 'TMV_ms': 109029, 'TT_ms': 1359959, 'DL_ms': '---', 'DA_ms': '---', 'QUALIFICADO': True, 'MVC': False, 'MAPA_VOLTAS': ['01:51.397', '01:52.454', '01:55.851', '01:52.474', '01:50.899', '01:54.426', '01:49.934', '02:04.787', '01:49.029', '01:57.917', '01:49.157', '01:51.634'], 'MAPA_VOLTAS_ms': [111397, 112454, 115851, 112474, 110899, 114426, 109934, 124787, 109029, 117917, 109157, 111634], 'MAPA_VOLTAS_ACC_ms': [111397, 223851, 339702, 452176, 563075, 677501, 787435, 912222, 1021251, 1139168, 1248325, 1359959], 'MAPA_VOLTAS_ACC': ['01:51.397', '03:43.851', '05:39.702', '07:32.176', '09:23.075', '11:17.501', '13:07.435', '15:12.222', '17:01.251', '18:59.168', '20:48.32

TypeError: 'NoneType' object is not subscriptable

In [ ]:
DATA

'2024-09-25'

In [ ]:
file_name = f"../data/ranking/{DATA.replace('-', '')}.json"

with open(file_name, 'w') as file:
    json.dump(ranking, file, indent=4)

print(f"O arquivo {file_name} foi salvo com sucesso.")

O arquivo ../data/ranking/20240925.json foi salvo com sucesso.
